In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
standings_url = 'https://docs.google.com/spreadsheets/d/1r4Cqm5E8pyER3nXix_A05SD3-EMusJZtk3XZAeFexvg/pub?output=csv'
prices_url = 'https://docs.google.com/spreadsheets/d/1b5486fzknvQgXJxPuS7U88HwMYLrz-EfdfTO3-w5qx0/pub?output=csv'

wikipedia_f1_url = 'https://en.wikipedia.org/wiki/2022_Formula_One_World_Championship'

# F1 GP names

In [3]:
# Get F1 GP names from WDC standings Wikipedia table
df_col_names = pd.read_html(wikipedia_f1_url)[5].columns.values[2:-1]
df_col_names = np.insert(df_col_names, 0, 'Name', axis=0)
df_col_names

array(['Name', 'BHR', 'SAU', 'AUS', 'EMI', 'MIA', 'ESP', 'MON', 'AZE',
       'CAN', 'GBR', 'AUT', 'FRA', 'HUN', 'BEL', 'NED', 'ITA', 'SIN',
       'JPN', 'USA', 'MXC', 'SAP', 'ABU'], dtype=object)

# Standings Dataframe

In [4]:
def clean_standings(csv: pd.DataFrame, r1=0, r2=32):
    # Crop csv to only relevant section
    df = csv.iloc[r1:r2, :23]

    # Rename columns to Wikipedia GP names
    df.columns = df_col_names

    # Cast numeric columns
    df = df.apply(pd.to_numeric, errors='ignore')

    # Only keep finished races
    df = df.loc[:, ~(df == 0).all()]

    return df

In [5]:
standings_csv = pd.read_csv(standings_url)

In [6]:
standings_drvs = clean_standings(standings_csv, r2=20)
# Only keep driver last names (L.Hamilton -> Hamilton)
standings_drvs['Name'] = standings_drvs['Name'].apply(lambda x: re.sub('.*\.', '', x))
standings_drvs

,Name,BHR,SAU,AUS,EMI,MIA,ESP
0,Hamilton,34,13,26,5,19,21
1,Russell,28,26,33,33,36,33
2,Leclerc,49,41,49,30,50,5
3,Sainz,32,27,-5,7,31,25
4,Verstappen,5,45,4,62,56,45
5,Perez,4,22,35,44,28,49
6,Ricciardo,13,-8,18,5,8,2
7,Norris,1,22,24,43,-2,16
8,Vettel,4,15,-7,24,-5,17
9,Stroll,15,9,15,15,11,8


In [7]:
standings_mkrs = clean_standings(standings_csv, r1=21)
standings_mkrs

,Name,BHR,SAU,AUS,EMI,MIA,ESP
21,Mercedes,57,34,64,31,40,49
22,Ferrari,76,63,44,30,56,30
23,Red Bull,4,62,39,99,69,94
24,Mclaren,9,9,37,41,1,13
25,Aston Martin,14,19,3,32,1,20
26,Alpine,24,13,10,8,13,33
27,AlphaTauri,10,3,8,36,-3,15
28,Williams,15,-1,8,17,26,14
29,Alfa Romeo,31,3,24,35,3,10
30,Haas,27,1,13,8,5,-6


# Price Dataframe

In [8]:
def clean_prices(csv: pd.DataFrame, r1=0, r2=30):
    df = csv[['Name', 'SeasonStartPrice', 'CurrentPrice']]
    df = df.iloc[r1:r2, :]

    # Cast numeric columns
    df = df.apply(pd.to_numeric, errors='ignore')

    return df

In [9]:
prices_csv = pd.read_csv(prices_url)

In [10]:
prices_drvs = clean_prices(prices_csv, r2=20)
prices_drvs

,Name,SeasonStartPrice,CurrentPrice
0,Hamilton,31.0,30.1
1,Russell,24.0,23.9
2,Leclerc,18.0,19.0
3,Sainz,17.0,17.2
4,Verstappen,30.5,30.3
5,Perez,17.5,18.5
6,Ricciardo,14.5,13.7
7,Norris,16.0,15.9
8,Vettel,11.5,11.4
9,Stroll,9.5,8.9


In [11]:
prices_mkrs = clean_prices(prices_csv, r1=20)
prices_mkrs

,Name,SeasonStartPrice,CurrentPrice
20,Mercedes,34.5,33.8
21,Ferrari,25.0,25.7
22,Red Bull,32.5,32.8
23,Mclaren,18.5,17.6
24,Aston Martin,11.5,10.9
25,Alpine,14.0,13.9
26,AlphaTauri,10.5,10.0
27,Williams,7.0,6.5
28,Alfa Romeo,8.0,8.6
29,Haas,6.0,6.4


# Turbo Driver

In [12]:
# Filter drivers to those worth less than 20M
eligible_turbo = prices_drvs[prices_drvs.CurrentPrice < 20]['Name'].to_list()
eligible_turbo

['Leclerc',
 'Sainz',
 'Perez',
 'Ricciardo',
 'Norris',
 'Vettel',
 'Stroll',
 'Alonso',
 'Ocon',
 'Gasly',
 'Tsunoda',
 'Albon',
 'Latifi',
 'Bottas',
 'Zhou',
 'Schumacher',
 'Magnussen']

In [13]:
turbo_standings = standings_drvs.copy()

# Filter to only turbo drivers
turbo_standings = turbo_standings[turbo_standings['Name'].isin(eligible_turbo)]

# Apply x2 bonus
turbo_standings = turbo_standings.apply(lambda col: col * 2 if col.name != 'Name' else col)
turbo_standings

,Name,BHR,SAU,AUS,EMI,MIA,ESP
2,Leclerc,98,82,98,60,100,10
3,Sainz,64,54,-10,14,62,50
5,Perez,8,44,70,88,56,98
6,Ricciardo,26,-16,36,10,16,4
7,Norris,2,44,48,86,-4,32
8,Vettel,8,30,-14,48,-10,34
9,Stroll,30,18,30,30,22,16
10,Alonso,18,-6,-10,4,10,28
11,Ocon,40,42,40,26,26,48
12,Gasly,-8,34,28,26,-2,10


# Knapsack solver

In [14]:
# https://www.geeksforgeeks.org/extended-knapsack-problem/
def knapsack(vals, weights, n, max_W, max_E):
    dp = [[[0 for _ in range(max_E + 1)]for _ in range(max_W + 1)] for _ in range(n+1)]
    drivers = [[[[] for _ in range(max_E + 1)]for _ in range(max_W + 1)] for _ in range(n+1)]

    # for each element given
    for i in range(1, n+1):
 
        # For each possible
        # weight value
        for j in range(1, max_W+1):
 
            # For each case where
            # the total elements are
            # less than the constraint
            for k in range(1, max_E+1):
 
                # To ensure that we dont
                # go out of the array
                if (j >= weights[i - 1]):
                    no_change = dp[i - 1][j][k]
                    with_change = dp[i - 1][j - weights[i - 1]][k - 1] + vals[i - 1]

                    if with_change > no_change:
                        dp[i][j][k] = with_change
                        drivers[i][j][k] = drivers[i - 1][j - weights[i - 1]][k - 1] + [i - 1]
                    else:
                        dp[i][j][k] = no_change
                        drivers[i][j][k] = drivers[i - 1][j][k]
                 
                else:
                    dp[i][j][k] = dp[i - 1][j][k]
                    drivers[i][j][k] = drivers[i - 1][j][k]
 
    return dp[n][max_W][max_E], drivers[n][max_W][max_E]

In [15]:
# For every Gran Prix
for gp in standings_drvs.columns.values[1:]:
    best_mkr = ''
    best_drivers = ''
    best_score = 0

    # For every eligible Turbo Driver
    for turbo_drv in eligible_turbo:
        # Double their score and store cost
        turbo_score = turbo_standings.loc[turbo_standings.Name == turbo_drv][gp].item()
        turbo_cost = prices_drvs.loc[prices_drvs.Name == turbo_drv].CurrentPrice

        # Remove them from no-bonus scoreboard for knapsack solver
        remaining_drvs = standings_drvs.loc[standings_drvs.Name != turbo_drv]
        # Join points and prices dataframes
        remaining_drvs = pd.merge(remaining_drvs, prices_drvs, on='Name')

        # For every Constructor
        for _, row in pd.merge(standings_mkrs, prices_mkrs, on="Name").iterrows():
            # Extract info from Constructor
            mkr_cost = row.CurrentPrice
            mkr_score = row[gp]
            mkr_name = row['Name']

            # Extract data for knapsack problem
            vals = remaining_drvs[gp].to_list()                                 # Pts scored
            n = len(vals)                                                       # Number of drivers
            weights = (10 * remaining_drvs.CurrentPrice).astype(int).to_list()  # Driver cost (x10 to remove dec. part)
            max_W = int(10 * (100 - mkr_cost - turbo_cost))                     # Budget rem. after Constructor and Turbo Driver
            max_E = 4                                                           # Maximum of 4 drivers (5 drivers - Turbo)

            drvs_score, drivers = knapsack(vals, weights, n, max_W, max_E)
            total_score = drvs_score + mkr_score + turbo_score

            if total_score > best_score:
                best_mkr = mkr_name
                best_score = total_score
                best_drivers = [turbo_drv + ' (TD)'] + remaining_drvs.Name.iloc[drivers].tolist()

    print(f'{gp} ({best_score}): {best_mkr:>12}\t+ {", ".join(best_drivers)}')

BHR (276):      Ferrari	+ Leclerc (TD), Hamilton, Tsunoda, Bottas, Magnussen
SAU (231):      Ferrari	+ Leclerc (TD), Sainz, Verstappen, Magnussen
AUS (249):     Mercedes	+ Leclerc (TD), Perez, Ocon, Bottas, Schumacher
EMI (318):     Red Bull	+ Perez (TD), Norris, Vettel, Tsunoda, Bottas
MIA (261):     Red Bull	+ Leclerc (TD), Verstappen, Albon, Bottas
ESP (279):     Red Bull	+ Perez (TD), Sainz, Ocon, Tsunoda, Bottas
